In [3]:
import sys
sys.path.append('../_amigocloud')

In [4]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

import pandas as pd
import json

In [5]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
# conexion a amigocloud
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [18]:
fecha = '2024-03-23'
query_dia = {'query': 'select id, canhero from dataset_289611 where date(fecha_registro)=\'{fech}\''.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select_dia = amigocloud.get(project, query_dia)
data_dia = select_dia['data']
data_dia

[{'id': 48, 'canhero': '515 / AGUILERA OLGA RIVERO DE'},
 {'id': 49, 'canhero': '805 / AGROP. TRES HNOS DEL ORIENTE'}]

In [19]:
select_dia

{'query': "select id, canhero from dataset_289611 where date(fecha_registro)='2024-03-23'",
 'count': 2,
 'limit': 100,
 'offset': 0,
 'columns': [{'type': 'integer', 'name': 'id'},
  {'type': 'string', 'name': 'canhero'}],
 'data': [{'id': 48, 'canhero': '515 / AGUILERA OLGA RIVERO DE'},
  {'id': 49, 'canhero': '805 / AGROP. TRES HNOS DEL ORIENTE'}],
 'next': None}

In [20]:
for insp in data_dia:
    
    print('Generando reporte de:', insp['canhero'], '------------------------------------------')
    id_insp = insp['id']
    
    # consulta para buscar por id
    query = {'query': 'select * from dataset_289611 where id={id}'.format(id=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    # cambio del formato de fecha
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    # convercion a objeto
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # consulta para buscar 
    query = {'query': 'select * from dataset_289610 where calidad_ref_muestra=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
    select = amigocloud.get(project, query)
    data_muestras = select['data']
    
    # calculo de promedio de muestras
    keys_to_avg = ['profundidad_surco', 
               'distancia_entre_surco', 
               'tamaño_de_cañoto', 
               'cantidad_tierra', 
               'falla_de_tapado', 
               'peso_en_5m', 
               'cantidad_de_semilla', 
               'yemas_metro', 
               'yemas_no_viables']
    averages = {}
    for key in keys_to_avg:
        total = sum(d[key] for d in data_muestras)
        averages[key] = total / len(data_muestras)
    averages['yemas_metro'] = int(averages['yemas_metro']) # redondea
    data = collections.namedtuple("data", averages.keys())(*averages.values())
    
    # get fotos
    query_fotos = {'query': 'select insp.lote, muestra.obs, gal.s3_filename from dataset_289611 insp inner join dataset_289610 muestra on insp.amigo_id=muestra.calidad_ref_muestra inner join gallery_41234 gal on muestra.amigo_id=gal.source_amigo_id where insp.amigo_id=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
    select_fotos = amigocloud.get(project, query_fotos)
    fotos = select_fotos['data']
    
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_calidad-siembra.docx")
    
    # SE DESCARGAN LAS FOTOS
    lista_fotos_inline = []
    i = 1
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['s3_filename'], width=Mm(60)), 
            'muestra': 'M' + str(i),
            'obs':foto['obs']
        })
        i=i+1
    
    # estableser firma de tecnico
    firma_respon = None
    if insp.usuario == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_jaldin.png', width=Mm(60))
        
    # nombre de archivo
    filename = '/_' + insp.canhero.split(' / ')[0] + '_CDS_' + insp.fecha_registro.replace('/','-') + '_' + insp.canhero.split(' / ')[1] + '_' + str(insp.id) + '.docx'
    
    # CREAR EL CONTEXTO DEL DOC Y CREAR EL DOC
    context = {'insp':insp, 'data':data, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    doc.save(ruta + '/informes' + filename)
    print('Trarea terminada.')
print('END=========================================')

Generando reporte de: 515 / AGUILERA OLGA RIVERO DE ------------------------------------------
Trarea terminada.
Generando reporte de: 805 / AGROP. TRES HNOS DEL ORIENTE ------------------------------------------
Trarea terminada.
END=========================================


## Convertir a PDF

In [8]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 1 INFORMES
1 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/40256_CDS_07-06-2023_INSERVA DAMIANA S.R.L._35.docx']

In [9]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\pdf\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/40256_CDS_07-06-2023_INSERVA DAMIANA S.R.L._35.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/40256_CDS_07-06-2023_INSERVA DAMIANA S.R.L._35.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/40256_CDS_07-06-2023_INSERVA DAMIANA S.R.L._35.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END
